### <b> 전략 <b>
1. 가치주 : 일자 기준 저pbr, 저per (단, per>0이고 pbr>0인 종목)
2. 수익성 : 전분기 eps 변화율이 높은 종목을 찾습니다. 
3. 안정성 : 부채비율 변화율<0인 종목을 찾습니다.
4. 이격도 리버전 : 단기 투자 전략에 맞게 한달 간 이격도가 낮은 종목을 고릅니다.

In [10]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)
import json
import requests

from pykrx import stock

In [63]:
#업데이트의 편의를 위해 api 사용
data = stock.get_market_cap("20240319", market="ALL")
data = data[['종가', '시가총액']]
data.head()

,종가,시가총액
티커,,
005930,72800,434600169640000
000660,160200,116625978873000
373220,403000,94302000000000
207940,848000,60355552000000
005935,61800,50854398060000


### 1.1 가치주 : 저per, 저pbr, 

In [64]:
#api 활용 데이터 - per, pbr
df = stock.get_market_fundamental_by_ticker(date='20240319', market="ALL")
df.head()

,BPS,PER,PBR,EPS,DIV,DPS
티커,,,,,,
060310,947,106.33,3.37,30,0.00,0
095570,8076,24.50,0.61,201,5.48,270
006840,41948,0.00,0.37,0,1.29,200
054620,10864,14.57,0.68,505,0.00,0
265520,17980,3.72,1.13,5463,1.33,270


In [65]:
df.describe() #per<5 고정 #PBR의 경우 상위 25% 값보다 작은 것

,BPS,PER,PBR,EPS,DIV,DPS
count,2.622000e+03,2622.000000,2622.000000,2622.000000,2622.000000,2622.000000
mean,2.195963e+04,32.011850,2.345397,1718.613654,1.280561,343.768879
std,1.204684e+05,278.339978,11.258188,10138.308960,2.135117,1160.520751
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.452750e+03,0.000000,0.480000,0.000000,0.000000,0.000000
50%,4.738000e+03,5.700000,0.960000,143.000000,0.000000,0.000000
75%,1.258025e+04,17.295000,2.020000,938.500000,1.947500,200.000000
max,4.906107e+06,9342.860000,498.530000,406090.000000,37.100000,20000.000000


In [66]:
#업데이트 편의를 위해 넘파이 사용
import numpy as np
column_pbr = df['PBR'].values
pbr_25 = np.percentile(column_pbr, 25)

df2 = df.loc[(df['PER'] < 5) & (df['PBR'] < pbr_25)] #저per, 저pbr인 데이터
df2 = df2[['PER', 'PBR']]
df2

,PER,PBR
티커,,
006840,0.00,0.37
138930,3.30,0.26
001465,0.00,0.00
035760,0.00,0.44
00104K,0.00,0.00
...,...,...
028080,0.00,0.45
079980,0.00,0.34
005010,0.89,0.31


In [67]:
df3 = pd.merge(data, df2, on='티커')
df3

,종가,시가총액,PER,PBR
티커,,,,
005935,61800,50854398060000,0.00,0.00
015760,22400,14379995324800,0.00,0.35
024110,14700,11722160274300,4.43,0.40
316140,14660,11023579098260,3.50,0.37
011200,15680,10804139297280,0.76,0.37
...,...,...,...,...
001527,11500,3548543500,0.00,0.00
021045,8300,3521241800,0.00,0.00
001525,5690,3514889390,0.00,0.00


### 1.2 PER > 0, PBR > 0

In [68]:
df3 = df3.loc[(df['PER'] > 0) & (df['PBR'] > 0)]
df3 = df3[['종가', 'PER', 'PBR']]
df3.shape #per>0 이고 pbr>0인 65개의 항목을 추출합니다

(141, 3)

In [169]:
df3.index

Index(['024110', '316140', '011200', '078930', '004020', '088350', '138930',
       '036460', '000880', '139480',
       ...
       '000950', '012620', '050120', '225590', '054940', '377460', '001770',
       '073540', '106240', '115570'],
      dtype='object', name='티커', length=141)

### 2. 부채비율과 고EPS 변화율
- 부채비율 : 직전 분기 활용
- 고eps 변화율 : 1m

In [127]:
#추가적인 정보를 가져오기 위해 requests 라이브러리를 활용하였습니다.
ticker = 290120
url = 'https://navercomp.wisereport.co.kr/v2/company/ajax/cF1001.aspx?cmp_cd=316140&fin_typ=4&freq_typ=Q&encparam=ZXRYWU1WeWJMNjFpU3ZSUUhob2lzUT09&id=Qk80WlNFN0'
headers= {
'Referer':'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=290120&target=finsum_more',
'User-Agent' :
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
'X-Requested-With' :
'XMLHttpRequest' }

res = requests.get(url=url, headers=headers)
df = pd.read_html(res.text)[1]
df

주요재무정보                분기                                      \
        주요재무정보 2022/12  (IFRS연결) 2023/03  (IFRS연결) 2023/06  (IFRS연결)   
0          매출액     -7.472000e+03      1.252940e+05      1.082790e+05   
1         영업이익      7.279000e+03      1.252000e+04      8.954000e+03   
2   영업이익(발표기준)      7.279000e+03      1.252000e+04      8.954000e+03   
3     세전계속사업이익      7.293000e+03      1.270300e+04      8.782000e+03   
4        당기순이익      5.313000e+03      9.466000e+03      6.672000e+03   
5    당기순이익(지배)      4.799000e+03      9.137000e+03      6.252000e+03   
6   당기순이익(비지배)      5.140000e+02      3.290000e+02      4.200000e+02   
7         자산총계      4.804744e+06      4.780793e+06      4.829116e+06   
8         부채총계      4.488471e+06      4.454756e+06      4.502204e+06   
9         자본총계      3.162730e+05      3.260360e+05      3.269130e+05   
10    자본총계(지배)      2.876190e+05      2.965970e+05      3.024470e+05   
11   자본총계(비지배)      2.865400e+04      2.943900e+04      2.446600e+04   
12         자본금      3.640300e+04      3.640300e+04      3.640300e+04   
13    영업활동현금흐름      2.439840e+05     -3.451500e+04     -7.447600e+04   
14    투자활동현금흐름     -3.880500e+04      3.922000e+03     -3.445000e+03   
15    재무활동현금흐름     -6.863100e+04     -3.368000e+03     -1.366200e+04   
16       CAPEX      6.070000e+02      2.230000e+02      3.150000e+02   
17         FCF               NaN               NaN               NaN   
18      이자발생부채      6.998380e+05      7.077720e+05      7.091020e+05   
19       영업이익률     -9.741000e+01      9.990000e+00      8.270000e+00   
20        순이익률     -7.110000e+01      7.560000e+00      6.160000e+00   
21      ROE(%)      1.151000e+01      1.150000e+01      1.019000e+01   
22      ROA(%)      7.200000e-01      7.200000e-01      6.400000e-01   
23        부채비율      1.419170e+03      1.366340e+03      1.377190e+03   
24       자본유보율      7.689200e+02      7.957300e+02      7.987900e+02   
25      EPS(원)      6.590000e+02      1.255000e+03      8.590000e+02   
26      PER(배)      2.680000e+00      2.580000e+00      2.940000e+00   
27      BPS(원)      3.950500e+04      4.073800e+04      4.165500e+04   
28      PBR(배)      2.900000e-01      2.800000e-01      2.800000e-01   
29    현금DPS(원)      9.800000e+02      0.000000e+00      1.800000e+02   
30     현금배당수익률      8.480000e+00      0.000000e+00      1.530000e+00   
31   현금배당성향(%)      1.486700e+02      0.000000e+00      2.091000e+01   
32  발행주식수(보통주)      7.280605e+08      7.280605e+08      7.280605e+08   

                                                             \
   2023/09  (IFRS연결) 2023/12  (IFRS연결) 2024/03(E)  (IFRS연결)   
0       9.884000e+04      8.747300e+04                  NaN   
1       1.218200e+04      1.333000e+03              11434.0   
2       1.218200e+04      1.333000e+03                  NaN   
3       1.246800e+04      1.221000e+03              11560.0   
4       9.181000e+03      9.500000e+02               8720.0   
5       8.993000e+03      6.810000e+02               8213.0   
6       1.880000e+02      2.690000e+02                  NaN   
7       4.859170e+06      4.980049e+06                  NaN   
8       4.526956e+06      4.646074e+06                  NaN   
9       3.322140e+05      3.339750e+05                  NaN   
10      3.158520e+05      3.166690e+05                  NaN   
11      1.636200e+04      1.730600e+04                  NaN   
12      3.802700e+04      3.802700e+04                  NaN   
13      6.193600e+04      6.327500e+04                  NaN   
14     -1.490900e+04     -8.110000e+03                  NaN   
15     -5.379000e+03     -6.193000e+03                  NaN   
16      3.140000e+02      7.950000e+02                  NaN   
17               NaN               NaN                  NaN   
18      7.059820e+05      7.001440e+05                  NaN   
19      1.232000e+01      1.520000e+00                  NaN   
20      9.290000e+00      1.090000e+00                  NaN   
21      9.750000e+00      8.290000e+00

In [128]:
df.columns

MultiIndex([('주요재무정보',               '주요재무정보'),
            (    '분기',    '2022/12  (IFRS연결)'),
            (    '분기',    '2023/03  (IFRS연결)'),
            (    '분기',    '2023/06  (IFRS연결)'),
            (    '분기',    '2023/09  (IFRS연결)'),
            (    '분기',    '2023/12  (IFRS연결)'),
            (    '분기', '2024/03(E)  (IFRS연결)'),
            (    '분기', '2024/06(E)  (IFRS연결)'),
            (    '분기', '2024/09(E)  (IFRS연결)')],
           )

In [131]:
import pandas as pd
import json
import requests

# 분석할 종목들
tickers = df3.index

# 결과를 저장할 데이터프레임
result_df = pd.DataFrame(columns=['종목', '변화율'])

# 각 종목에 대해 반복하여 데이터 수집 및 분석 수행
for ticker in tickers:
    # URL 및 Referer 생성
    url_template = 'https://navercomp.wisereport.co.kr/v2/company/ajax/cF1001.aspx?cmp_cd={}&fin_typ=4&freq_typ=Q&encparam=ZXRYWU1WeWJMNjFpU3ZSUUhob2lzUT09&id=Qk80WlNFN0'
    url = url_template.format(ticker)
    referer_template = 'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=290120&target=finsum_more'
    referer = referer_template.replace('290120', ticker)
    
    # HTTP 요청
    headers = {
        'Referer': referer,
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest'
    }
    res = requests.get(url=url, headers=headers)
    df = pd.read_html(res.text)[1]
    df = df[[('주요재무정보', '주요재무정보'),
           (    '분기',    '2023/06  (IFRS연결)'),
            (    '분기',    '2023/09  (IFRS연결)')]]

    df = df.iloc[[13, 23, 24, 25, 32]]
    df['변화율'] = (df.iloc[:, 2] - df.iloc[:, 1]) / df.iloc[:, 1]  
    
    # 종목별 결과를 데이터프레임에 추가
    df['종목'] = ticker   # '종목' 열에 티커 값 할당
    result_df = pd.concat([result_df, df], ignore_index=True)

result_df

,종목,변화율,"(주요재무정보, 주요재무정보)","(분기, 2023/06 (IFRS연결))","(분기, 2023/09 (IFRS연결))","(변화율, )","(종목, )"
0,NaN,NaN,영업활동현금흐름,1.021000e+03,-6.393900e+04,-63.623898,024110
1,NaN,NaN,부채비율,1.353000e+03,1.314750e+03,-0.028271,024110
2,NaN,NaN,자본유보율,6.225700e+02,6.384700e+02,0.025539,024110
3,NaN,NaN,EPS(원),8.330000e+02,9.140000e+02,0.097239,024110
4,NaN,NaN,발행주식수(보통주),7.974259e+08,7.974259e+08,0.000000,024110
...,...,...,...,...,...,...,...
700,NaN,NaN,영업활동현금흐름,1.400000e+01,-1.200000e+01,-1.857143,115570
701,NaN,NaN,부채비율,4.127000e+01,4.080000e+01,-0.011388,115570
702,NaN,NaN,자본유보율,1.939480e+03,1.913340e+03,-0.013478,115570
703,NaN,NaN,EPS(원),-6.100000e+01,-1.310000e+02,1.147541,115570


In [132]:
result_df.columns

Index([                       '종목',                       '변화율',
              ('주요재무정보', '주요재무정보'), ('분기', '2023/06  (IFRS연결)'),
       ('분기', '2023/09  (IFRS연결)'),                 ('변화율', ''),
                        ('종목', '')],
      dtype='object')

In [133]:
grouped_df = result_df.groupby(('종목', ''))
grouped_df.head()

,종목,변화율,"(주요재무정보, 주요재무정보)","(분기, 2023/06 (IFRS연결))","(분기, 2023/09 (IFRS연결))","(변화율, )","(종목, )"
0,NaN,NaN,영업활동현금흐름,1.021000e+03,-6.393900e+04,-63.623898,024110
1,NaN,NaN,부채비율,1.353000e+03,1.314750e+03,-0.028271,024110
2,NaN,NaN,자본유보율,6.225700e+02,6.384700e+02,0.025539,024110
3,NaN,NaN,EPS(원),8.330000e+02,9.140000e+02,0.097239,024110
4,NaN,NaN,발행주식수(보통주),7.974259e+08,7.974259e+08,0.000000,024110
...,...,...,...,...,...,...,...
700,NaN,NaN,영업활동현금흐름,1.400000e+01,-1.200000e+01,-1.857143,115570
701,NaN,NaN,부채비율,4.127000e+01,4.080000e+01,-0.011388,115570
702,NaN,NaN,자본유보율,1.939480e+03,1.913340e+03,-0.013478,115570
703,NaN,NaN,EPS(원),-6.100000e+01,-1.310000e+02,1.147541,115570


In [134]:
df = result_df.drop(columns=['종목', '변화율'])
df.columns.to_list()

[('주요재무정보', '주요재무정보'),
 ('분기', '2023/06  (IFRS연결)'),
 ('분기', '2023/09  (IFRS연결)'),
 ('변화율', ''),
 ('종목', '')]

In [135]:
df = df.rename(columns={('주요재무정보', '주요재무정보'): '재무비율', 
                        ('연간', '2020/12  (IFRS연결)') : '2020/12',
                        ('연간', '2021/12  (IFRS연결)') : '2021/12',
                        ('연간', '2022/12  (IFRS연결)') : '2022/12',
                        ('변화율', ''): '변화율', 
                        ('종목', ''): '종목'})

In [136]:
df.columns

Index(['재무비율', ('분기', '2023/06  (IFRS연결)'), ('분기', '2023/09  (IFRS연결)'), '변화율',
       '종목'],
      dtype='object')

In [137]:
df

,재무비율,"(분기, 2023/06 (IFRS연결))","(분기, 2023/09 (IFRS연결))",변화율,종목
0,영업활동현금흐름,1.021000e+03,-6.393900e+04,-63.623898,024110
1,부채비율,1.353000e+03,1.314750e+03,-0.028271,024110
2,자본유보율,6.225700e+02,6.384700e+02,0.025539,024110
3,EPS(원),8.330000e+02,9.140000e+02,0.097239,024110
4,발행주식수(보통주),7.974259e+08,7.974259e+08,0.000000,024110
...,...,...,...,...,...
700,영업활동현금흐름,1.400000e+01,-1.200000e+01,-1.857143,115570
701,부채비율,4.127000e+01,4.080000e+01,-0.011388,115570
702,자본유보율,1.939480e+03,1.913340e+03,-0.013478,115570
703,EPS(원),-6.100000e+01,-1.310000e+02,1.147541,115570


In [199]:
df['재무비율'].value_counts()

영업활동현금흐름      141
부채비율          141
자본유보율         141
EPS(원)        141
발행주식수(보통주)    141
Name: 재무비율, dtype: int64

In [200]:
#부채비율이 전분기에 비해 감소한 기업
df.loc[(df['재무비율'] == '부채비율') & (df['변화율'] < 0)]['종목'].unique()
temp1 = df.loc[(df['재무비율'] == '부채비율') & (df['변화율'] < 0)]
temp1.shape 

(74, 5)

In [201]:
df.loc[(df['재무비율'] == 'EPS(원)')]['변화율'].describe()

count    124.000000
mean       1.480111
std       15.812394
min       -7.620421
25%       -0.881541
50%       -0.433455
75%        0.221365
max      168.636364
Name: 변화율, dtype: float64

In [202]:
#고EPS 
df.loc[(df['재무비율'] == 'EPS(원)') & (df['변화율'] > 0)]['종목'].unique()
temp2 = df.loc[(df['재무비율'] == 'EPS(원)') & (df['변화율'] > 0)]
temp2.shape 

(41, 5)

In [203]:
#저per이고, 부채비율이 감소하였으면서, eps가 증가한 종목
final = pd.merge(temp1, temp2, on='종목')
final['종목'] #재무비율 활용 끝

0     024110
1     316140
2     078930
3     023590
4     006120
5     016380
6     007700
7     032190
8     123890
9     093050
10    005010
11    007860
12    023600
13    014280
14    000320
15    000700
16    037460
17    005710
18    009070
19    017650
20    129260
21    155660
22    290120
23    050120
24    115570
Name: 종목, dtype: object

### 3. 이격도 리버전 (1m)

In [146]:
ticker_list = final['종목'].tolist()
ticker_list

['024110',
 '316140',
 '078930',
 '023590',
 '006120',
 '016380',
 '007700',
 '032190',
 '123890',
 '093050',
 '005010',
 '007860',
 '023600',
 '014280',
 '000320',
 '000700',
 '037460',
 '005710',
 '009070',
 '017650',
 '129260',
 '155660',
 '290120',
 '050120',
 '115570']

In [148]:
#이격도가 이동평균선보다 낮은 티커 필터링
#날짜에 따라 이격도가 다를 수 있음
result_df = pd.DataFrame()
tickers = ticker_list

for ticker in tickers : 
    data = stock.get_market_ohlcv('20230320', '20240320', ticker)
    data = data[['종가']]
    std_dev = data['종가'].rolling(20).std() 
    
    data['중심'] = data['종가'].rolling(20).mean() 
    data['이격도'] = (data['종가'] / data['중심']) * 100
    data['상단'] = data['중심'] + 2 * data['종가'].rolling(window=20).std()
    data['하단'] = data['중심'] - 2 * data['종가'].rolling(window=20).std()
    
    data = data.dropna()
    
    result_df[ticker] = data['이격도']

result_df = result_df.T

In [56]:
#이격도가 이동평균선보다 낮은 티커 필터링
#날짜에 따라 이격도가 다를 수 있음
result_df = pd.DataFrame()
tickers = ticker_list

for ticker in tickers : 
    data = stock.get_market_ohlcv('20200320', '20240320', ticker)
    data = data[['종가']]
    std_dev = data['종가'].rolling(20).std() 
    
    data['중심'] = data['종가'].rolling(20).mean() 
    data['상단'] = data['중심'] + 2 * data['종가'].rolling(window=20).std()
    data['하단'] = data['중심'] - 2 * data['종가'].rolling(window=20).std()
    data['이격도'] = (data['종가'] / data['중심']) * 100
    
    data = data.dropna()
    result_df[ticker] = data['이격도']

result_df = result_df.T

In [204]:
data

,종가,중심,이격도,상단,하단
날짜,,,,,
2023-04-14,4590,4749.75,96.636665,5207.127824,4292.372176
2023-04-17,4515,4722.00,95.616264,5164.676073,4279.323927
2023-04-18,4630,4703.50,98.437334,5127.802524,4279.197476
2023-04-19,4620,4684.50,98.623119,5086.336573,4282.663427
2023-04-20,4485,4657.75,96.291128,5036.090907,4279.409093
...,...,...,...,...,...
2024-03-14,2555,2793.25,91.470509,3029.320351,2557.179649
2024-03-15,2555,2775.50,92.055485,3027.458643,2523.541357
2024-03-18,2550,2757.50,92.475068,3020.208564,2494.791436


In [205]:
data[['중심', '종가', '상단', '하단']].iplot(kind='scatter') #하락추세임을 확인

In [159]:
test = result_df.iloc[:, -30:] #직전 1개월 주가 이격도
test

날짜,2024-02-05,2024-02-06,2024-02-07,2024-02-08,2024-02-13,2024-02-14,2024-02-15,2024-02-16,2024-02-19,2024-02-20,...,2024-03-07,2024-03-08,2024-03-11,2024-03-12,2024-03-13,2024-03-14,2024-03-15,2024-03-18,2024-03-19,2024-03-20
024110,113.796702,110.001243,109.497758,109.451518,109.388284,106.655942,106.389776,108.656054,109.930607,106.262155,...,102.819480,103.292211,103.666521,106.577852,107.204611,112.126839,108.167536,106.247794,103.528417,103.948937
316140,112.569641,110.433630,108.993874,108.944484,106.498797,103.297108,103.428069,107.140263,108.372612,104.725506,...,99.007869,97.873361,97.683027,97.934253,100.123440,103.558986,103.644221,101.411325,99.846756,100.170242
078930,116.069337,113.592853,119.580780,117.547330,113.915568,110.067411,103.213776,105.823555,110.574342,106.068658,...,99.531377,99.251032,98.342202,96.394218,96.807521,102.805187,101.834673,101.174531,102.209374,104.991749
023590,122.186495,113.923042,114.888135,116.527077,124.360287,117.878688,114.097805,113.482842,112.835662,108.336921,...,102.150538,101.630556,98.883929,98.514464,101.143754,105.581708,100.964775,100.247247,99.188458,99.763006
006120,113.021475,110.974743,115.216833,113.943411,113.169375,110.574208,107.723957,107.961213,115.553389,113.118901,...,99.827692,98.346098,97.989112,96.149566,97.588147,100.578549,98.138372,97.878014,98.243658,98.243258
016380,103.681540,101.297177,102.145979,103.005411,102.547397,101.608806,101.672850,102.137613,101.892663,99.851065,...,93.347357,95.520813,96.272264,95.662586,95.460927,96.523023,97.439353,97.468526,95.834468,95.752684
007700,130.218232,111.989588,116.290307,118.302505,113.307481,108.478179,102.817383,105.230750,109.894122,106.448311,...,89.562723,92.480452,92.776791,93.582968,93.198159,94.414371,94.625743,94.532821,94.629470,97.181009
032190,108.650138,102.594167,106.917551,106.169610,107.553307,105.547550,102.344956,102.655753,104.634632,102.593130,...,95.248227,95.441140,93.955416,93.421005,94.056361,96.629949,98.078536,96.455251,96.515140,96.098989
123890,104.277438,104.089219,104.997635,104.990177,105.518591,102.748282,103.230829,103.598015,104.970670,104.656524,...,103.098883,103.082141,103.389955,100.477415,100.403670,101.742058,101.504308,99.642571,99.482168,99.197666
093050,106.647018,105.865965,105.868282,105.457482,105.961280,105.492323,102.857143,102.604997,105.255204,104.701417,...,100.716237,104.440759,103.709701,102.449070,103.104905,103.787434,103.544230,103.583038,103.903685,103.648092


In [163]:
quantiles = test.quantile(0.25)
mean = quantiles.mean()

mean #이격도 하위 값의 평균은 98입니다

98.05489396002623

In [164]:
#20240320 기준
final_df = result_df.loc[(result_df['2024-03-20'] < 98)] 
final_df.shape

(10, 228)

In [166]:
final_df.index #최종 스크리닝 종목

Index(['016380', '007700', '032190', '023600', '014280', '037460', '129260',
       '290120', '050120', '115570'],
      dtype='object')

### <b> 종목 정리 <b> 
    20240320 기준
    1. 016380 
    2. 007700
    3. 032190
    4. 023600
    5. 014280
    6. 037460
    7. 129260
    8. 290120
    9. 050120
    10. 115570